In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import random
# URL of the page
url = "https://www.facile.it/assicurazioni-auto/preventivo.html"
# url = "http://www.facile.it"
# Set up the Chrome driver (you need to download the ChromeDriver executable)
options = Options()

# options.add_argument("--headless")
options.add_argument("--disable-logging")
options.add_argument('--allow-running-insecure-content')
options.add_argument(f'--load-extension={"C:/Users/nicol/Downloads/www.facile.it.crt"}')
options.add_argument(f'Referer: {"https://www.facile.it/assicurazioni-auto/preventivo_2.html"}')

options.add_argument('--disable-application-cache')
options.add_argument('--disk-cache-size=1')
options.add_argument('--media-cache-size=1')
options.add_argument('--disable-browser-side-navigation')
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")

# options.add_argument("--proxy-server=http://your-proxy-server:123")
# Create a new instance of the Chrome driver
driver = webdriver.Chrome(options=options)
# Open the URL in the browser
driver.get(url)

In [2]:
payload ={
    "gender" : "Maschio",
    "birth_date": "07/07/1960",
    "birth_town": "Montichiari",
    "married": "S",
    "study_title": "bachelorDegree", #available options ["bachelorDegree", "masterDegree", "highSchool", "secondarySchool", "primarySchool", "none"]
    "job": "employee",
    "plate_number": 'FW085TZ',
    "purchase_year": "2019",
    "name": "Germana",
    "surname": "Tognon",
    "living_town": "Montichiari",
    "address": "Via XXV Aprile",
    "civic_number": "132",
    "email":"abrjhkhjukasa@gmail.com",
    "phone": "3398219745",
    "unsuspended_license" : "S"

    
}

In [3]:
def random_delay_wrapper(func):
    def wrapper(*args, **kwargs):

        delay = random.uniform(.5, 3)
        time.sleep(delay)
        
        result = func(*args, **kwargs)
        return result
    
    return wrapper

In [4]:
@random_delay_wrapper
def write_box(id_name: str, value: str, suggestion: bool =False):
    box_name = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, id_name))
    )
    box_name.clear()
    for char in value:
        box_name.send_keys(char)
        time.sleep(random.uniform(0.1, 0.3))
    if suggestion:
        time.sleep(random.uniform(0.5, 2))
        wait = WebDriverWait(driver, 3)

        first_suggestion = wait.until(
            EC.visibility_of_element_located((By.XPATH, '//div[@data-i="Autocomplete"]//div[contains(@class, "_1msz7s34")]/div[1]'))
        )

        # Click the first suggestion
        first_suggestion.click()
        # suggestions = WebDriverWait(driver, 10).until(
        #     EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".autocomplete-suggestions div"))
        # )
        # if suggestions:
        #     first_suggestion = suggestions[0]
        #     first_suggestion.click()

@random_delay_wrapper
def booelan_check_box(id_name: str, value: str):
    box_name = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f"//label[@for='{id_name}_{value}']"))
    )
    box_name.click()

@random_delay_wrapper
def select_box_by_index(id_name: str, index: int):
    dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, id_name))
    )

    select_index = Select(dropdown)
    select_index.select_by_index(index)

@random_delay_wrapper
def click_button(element_name:str):
    time.sleep(random.uniform(0.5, 2))
    driver.execute_script(f"document.getElementById('{element_name}').click();")

# Cooking starts here

In [5]:
cookie_button =WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//button[contains(text(), 'Continua senza accettare')]"))
)
cookie_button.click()

In [6]:
for radio_button in driver.find_elements(By.NAME, 'personType'):
    if radio_button.find_element(By.XPATH, './following-sibling::span').text == payload['gender']:
        driver.execute_script("arguments[0].click();", radio_button)
        break


In [7]:
write_box("birthDate", payload["birth_date"])
write_box("city",payload["birth_town"], suggestion=True)
for radio_button in driver.find_elements(By.NAME, 'marriedStatus'):
    if radio_button.find_element(By.XPATH, './following-sibling::span').text == payload['married']:
        driver.execute_script("arguments[0].click();", radio_button)
        break

In [8]:
time.sleep(random.uniform(0.5, 2))
dropdown = driver.find_element(By.NAME, 'educationalQualification')
Select(dropdown).select_by_value(payload['study_title'])

time.sleep(random.uniform(0.5, 2))
dropdown = driver.find_element(By.NAME, 'occupation')
Select(dropdown).select_by_value(payload['job'])


In [9]:
element = driver.find_element(By.NAME, 'drivingLicenseYear')
element.send_keys('1978')

In [10]:
driver.find_element(By.XPATH, "//button[@type='submit' and contains(@class, '_9h7zb28')]").click()

# First Page Done

In [ ]:
time.sleep(random.uniform(2, 3))
driver.get(driver.current_url)

In [ ]:
write_box("ASF_targa", payload["plate_number"])
write_box("ASF_acquisto_anno", payload["purchase_year"])
select_box_by_index("ASF_marca", 6)
select_box_by_index("ASF_modello",19)

In [ ]:
try:
    select_box_by_index("ASF_antifurto", 1)
except:
    print(f"ASF_antifurto not found")

In [ ]:
select_box_by_index("ASF_allestimento", 17)
select_box_by_index("ASF_ricovero", 1)
select_box_by_index("ASF_uso_veicolo", 2)
click_button("pulsanteAvanti")

# Second Page Done

In [ ]:
time.sleep(4)
driver.get(driver.current_url)

In [ ]:
write_box("ASF_decorrenza_giorno", "07")
write_box("ASF_decorrenza_mese", "01")
write_box("ASF_decorrenza_anno", "2024")
write_box("ASF_nome", payload["name"])
write_box("ASF_cognome", payload["surname"])

In [ ]:
click_button("retry_another_address")

write_box("ASF_comune_residenza", payload["living_town"], suggestion=True)
write_box("ASF_indirizzo",payload["address"])
write_box("ASF_civico",payload["civic_number"])
select_box_by_index("ASF_assi_provenienza", 2)
select_box_by_index("ASF_classe_merito", 2)
booelan_check_box("ASF_conducente_check", payload["unsuspended_license"])
select_box_by_index("ASF_eta_conducente_minore", 10)
write_box("ASF_email", payload["email"])
write_box("ASF_telefono", payload["phone"])
click_button("link_privacy_auth")
click_button("pulsanteAvanti")

# Preventivi

In [ ]:
time.sleep(25)

In [ ]:
driver.get(driver.current_url)

In [ ]:
html_content = driver.page_source

# Use Beautiful Soup to parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Find the element with the specified class name
soup = soup.find(class_="_1rf45hm8x lw2wjx0")

In [ ]:
targets = soup.findAll(class_="_1rf45hm8x lw2wjx0")

In [ ]:
preventivi = []
for preventivo in targets:
    text = preventivo.text.split(':')
    text = text[1:len(text)//2+1]
    prezzo = text[0].split('Compagnia')[0]
    if prezzo.split('€')[1] != '':
        prezzo = prezzo.split('€')[1] + '€'
    info_preventivo = {
        'Prezzo Annuo': prezzo,
        'Compagnia': text[1].split('Caratteristiche')[0],
        # 'Caratteristiche': text[2].split('Massimale')[0],
        # 'Massimale RC': text[3].split('Guida')[0]
    }
    preventivi.append(info_preventivo)


    


In [ ]:
preventivi
